In [2]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('SparkCleaning').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 16:22:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df_spark = spark.read.csv("../datasets/imdb/validation_hidden.csv", inferSchema=True, header=True)
df_spark.repartition(4)

DataFrame[_c0: int, tconst: string, primaryTitle: string, originalTitle: string, startYear: string, endYear: string, runtimeMinutes: string, numVotes: double]

In [5]:
df_spark.show()

23/03/09 16:22:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes
 Schema: _c0, tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes
Expected: _c0 but found: 
CSV file: file:///Users/mattiacintioli/PycharmProjects/Big-Data-Project/datasets/imdb/validation_hidden.csv
+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
|_c0|   tconst|        primaryTitle|      originalTitle|startYear|endYear|runtimeMinutes|numVotes|
+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
|  0|tt0003740|             Cabiria|               null|     1914|     \N|           148|  3452.0|
|  1|tt0008663|     A Man There Was|        Terje Vigen|     1917|     \N|            65|  1882.0|
|  3|tt0010307|           J'accuse!|               null|     1919|     \N|           166|  1692.0|

In [6]:
df_spark.count()

955

Starting to do stuff

In [7]:
import pandas as pd
import os

Count how many "train-i" files we have

In [8]:
dir_path = "../datasets/imdb/"

# Get a list of all the files in the directory
files = os.listdir(dir_path)

# Filter the list to include only files that start with "train-" and have a ".csv" extension
train_files = [f for f in files if f.startswith("train-") and f.endswith(".csv")]

# Count the number of files
num_train_files = len(train_files)

print("Number of train files: {}".format(num_train_files))

Number of train files: 8


print shape of training files, concatenate all the files, and print shape of big_file

In [9]:
# Create an empty list to store the DataFrames
dfs = []

# Loop through the file names and read each file into a DataFrame
for file in train_files:
    file_name = f"../datasets/imdb/{file}"
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    dfs.append(df)

# Loop through the DataFrames and print the shape of each one
for i, df in enumerate(dfs):
    num_rows = df.count()
    num_cols = len(df.columns)
    print("Shape of df_train_{}: {} rows, {} columns".format(i+1, num_rows, num_cols))

# Union all the DataFrames into one big DataFrame
big_df = dfs[0]
for df in dfs[1:]:
    big_df = big_df.union(df)
num_rows = big_df.count()
num_cols = len(big_df.columns)
print(f"\nShape of big_df: {num_rows} rows, {num_cols} columns")

Shape of df_train_1: 1001 rows, 9 columns
Shape of df_train_2: 1035 rows, 9 columns
Shape of df_train_3: 974 rows, 9 columns
Shape of df_train_4: 1049 rows, 9 columns
Shape of df_train_5: 963 rows, 9 columns
Shape of df_train_6: 951 rows, 9 columns
Shape of df_train_7: 993 rows, 9 columns
Shape of df_train_8: 993 rows, 9 columns

Shape of big_df: 7959 rows, 9 columns


Merging with writing.json

In [55]:
# Read the JSON file into a DataFrame
writing_df = spark.read.json("../datasets/imdb/writing.json")

# Rename the "movie" column in the JSON file to "tconst" to match the column name in big_df
writing_df = writing_df.withColumnRenamed("movie", "tconst")

# Join the two DataFrames based on the "tconst"
merged_df = big_df.join(writing_df, on='tconst', how='left')
merged_df.show()

23/03/09 17:13:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes, label
 Schema: _c0, tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes, label
Expected: _c0 but found: 
CSV file: file:///Users/mattiacintioli/PycharmProjects/Big-Data-Project/datasets/imdb/train-6.csv
+---------+---+---------------+--------------------+---------+-------+--------------+--------+-----+---------+
|   tconst|_c0|   primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|   writer|
+---------+---+---------------+--------------------+---------+-------+--------------+--------+-----+---------+
|tt0013257| 11|          Häxan|               Häxan|     1922|     \N|            91| 13679.0| true|nm0159725|
|tt0013556| 12|     Robin Hood|                null|     1922|     \N|           143|  2178.0| true|nm0940689|
|tt0013556| 12|     Robin Hood|   

Merging with directing.json

In [56]:
import json
with open("../datasets/imdb/directing.json") as f:
    data = json.load(f)

#convert to pandas
director_df = pd.DataFrame(data)

#convert to spark
director_df = spark.createDataFrame(director_df)

# Rename the "movie" column in the JSON file to "tconst"
director_df = director_df.withColumnRenamed("movie", "tconst")

# Join the two DataFrames based on the "tconst"
merged_df = merged_df.join(director_df, on='tconst', how='left')
merged_df.show()

23/03/09 17:13:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes, label
 Schema: _c0, tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes, label
Expected: _c0 but found: 
CSV file: file:///Users/mattiacintioli/PycharmProjects/Big-Data-Project/datasets/imdb/train-6.csv
+---------+---+---------------+---------------+---------+-------+--------------+--------+-----+---------+---------+
|   tconst|_c0|   primaryTitle|  originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|   writer| director|
+---------+---+---------------+---------------+---------+-------+--------------+--------+-----+---------+---------+
|tt0013257| 11|          Häxan|          Häxan|     1922|     \N|            91| 13679.0| true|nm0159725|nm0159725|
|tt0013556| 12|     Robin Hood|           null|     1922|     \N|           143|  2178.0| true|nm0940689|nm0245385|
|tt001355